In [2]:
#-------------------------------------------------------------------------
# Beam Design Optimization (Classical vs Quantum)
# Chapter 6 in the QUANTUM COMPUTING AND QUANTUM MACHINE LEARNING BOOK
#-------------------------------------------------------------------------
# Version 1.0
# (c) 2025 Jesse Van Griensven, Roydon Fraser, and Jose Rosas 
# Licence:  MIT - Citation of this work is required
#-------------------------------------------------------------------------
# Qiskit changes frequently. 
# We recommend using the latest version from the book code repository at:
# https://github.com/pedroer/quantum-computing-for-engineers/blob/main/requirements.txt
"""
Beam Design Optimization (Classical vs Quantum)

This script:
  - Uses classical optimization (COBYLA) to minimize cost.
  - Uses Qiskit's QAOA (Quantum Approximate Optimization Algorithm) for quantum optimization.
  - Converts the problem into a binary representation for QAOA compatibility.
"""

import numpy as np
import qiskit
import scipy
from scipy.optimize import minimize

# Check for Qiskit Optimization installation
try:
    from qiskit_optimization import QuadraticProgram
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    from qiskit.algorithms import QAOA
    from qiskit import Aer
    from qiskit.utils import QuantumInstance
    qiskit_optimization_available = True
except ImportError:
    qiskit_optimization_available = False

#-------------------------------------------------------------------------
# Print Library Versions
#-------------------------------------------------------------------------
print("\nLibrary Versions:")
print("Qiskit version:", qiskit.__version__)
print("NumPy version:", np.__version__)
print("SciPy version:", scipy.__version__)
print("Qiskit Optimization installed:", qiskit_optimization_available)

#-------------------------------------------------------------------------
# Define Problem Parameters
#-------------------------------------------------------------------------
weights = np.array([2, 3, 1, 4])    # Material weights
stresses = np.array([0.8, 1.2, 0.6, 1.5])  # Corresponding stresses

#-------------------------------------------------------------------------
# Classical Optimization (COBYLA)
#-------------------------------------------------------------------------
def beam_cost(params):
    """ Cost function for classical optimization """
    return np.sum(params * weights * stresses)
#-------------------------------------------------------------------------

# Classical optimization using COBYLA
initial_guess = [1, 1, 1, 1]
result_classical = minimize(beam_cost, initial_guess, method="COBYLA")
print("\nClassical Optimization Result:")
print("Optimized Material Distribution:", result_classical.x)
print("Minimum Cost:", result_classical.fun)

#-------------------------------------------------------------------------
# Quantum Optimization (QAOA via Qiskit), only if available
#-------------------------------------------------------------------------
if qiskit_optimization_available:
    # Define a Quadratic Program
    qp = QuadraticProgram()
    num_vars = len(weights)

    # **Binary Encoding: Convert Continuous Variables into Binary Variables**
    # - Each material variable x_i (originally continuous) is now represented by 3 binary bits (b_i1, b_i2, b_i3).
    # - The sum of these binary variables approximates the continuous value.
    bin_bits = 3  # Number of binary bits per variable
    scale_factor = 2**bin_bits - 1  # Scale continuous values to [0, 1] range

    for i in range(num_vars):
        for j in range(bin_bits):
            qp.binary_var(name=f"x{i}_b{j}")

    # **Define the objective function in a binary-compatible way**
    linear_objective = {}
    for i in range(num_vars):
        for j in range(bin_bits):
            coeff = (2**j) / scale_factor  # Scale binary representation
            linear_objective[f"x{i}_b{j}"] = coeff * weights[i] * stresses[i]

    qp.minimize(linear=linear_objective)  # Correct way to define the objective

    # Run QAOA Optimization
    backend = Aer.get_backend("aer_simulator")
    quantum_instance = QuantumInstance(backend)

    qaoa = QAOA(quantum_instance=quantum_instance, reps=3)  # Use 3 QAOA repetitions
    qaoa_optimizer = MinimumEigenOptimizer(qaoa)
    result_qaoa = qaoa_optimizer.solve(qp)

    #-------------------------------------------------------------------------
    # Compare Classical and Quantum Results
    #-------------------------------------------------------------------------
    print("\nQuantum Optimization Result (QAOA):")
    
    # Decode binary solution back to approximate continuous values
    qaoa_solution = []
    for i in range(num_vars):
        value = sum(result_qaoa.variables_dict[f"x{i}_b{j}"] * (2**j) / scale_factor for j in range(bin_bits))
        qaoa_solution.append(value)

    print("Optimized Material Distribution (QAOA Approximation):", qaoa_solution)
    print("Minimum Cost (QAOA):", result_qaoa.fval)
else:
    print("\nQiskit Optimization is not installed. Skipping QAOA optimization.")



Library Versions:
Qiskit version: 0.24.1
NumPy version: 1.23.5
SciPy version: 1.15.3
Qiskit Optimization installed: True

Classical Optimization Result:
Optimized Material Distribution: [-220.02585503 -496.30817407  -81.88469551 -827.84695732]
Minimum Cost: -7154.9633559369


/tmp/ipykernel_38999/1835217368.py:93: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend)
/tmp/ipykernel_38999/1835217368.py:95: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.qaoa.QAOA`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.QAOA``. See https://qisk.it/algo_migration for a migration guide.
  qaoa = QAOA(quantum_instance=quantum_instance, reps=3)  # Use 3 QAOA repetitions



Quantum Optimization Result (QAOA):
Optimized Material Distribution (QAOA Approximation): [0.2857142857142857, 0.0, 0.0, 0.0]
Minimum Cost (QAOA): 0.45714285714285713
